In [1]:
# 주피터노트북 목차 만들기
# 참고: https://mingchin.tistory.com/139

# pip install jupyter_contrib_nbextensions
# ! jupyter contrib nbextension install --user

# 목차
> ## 1. 사용자 평가 경향을 고려한(평점편차) 협업필터링
>> ### 1.1 데이터 읽어오고 정리하기
>> ### 1.2. def CF_knn_bias 설명

# # 1. 사용자 평가 경향을 고려한(평점편차) 협업필터링
<요약>  
사용자 평가 경향을 고려한 협업필터링은, KNN 이웃 기반 협업 필터링을 활용한 알고리즘이다.  
KNN 이웃 기반 협업필터링에서는  
user_id와 유사도가 높은 상위 [Neighbor size]명의, movie_id에 대한 **평점**에 **유사도**를 가중치 삼아 **가중 평균**으로 예상 평점을 계산했었다.  
  
사용자 평가 경향 고려 협업 필터링은 KNN 이웃 기반 협업 필터링을 조금 수정해  
평점 편차에 유사도를 가중치 삼아 평점 편차의 가중 평균을 구하고, 원하는 user_id의 전체 평균 평점에 더해지는 방식으로 예상 평점을 구한다.  
평점 편차의 개념은,  
**전체 평점 평균이 2점**인 사용자(이하 2점자)가 부여한 **3점**은 **평균**이라는 의미지만,  
**전체 평점 평균이 4점**인 사용자(이하 4점자)가 부여한 **3점**은 **보통 미만**의 의미가 된다는 것에서 시작된다.  
  
간단한 예시로  
'2점자'가 추천을 받은 user_id고, user_id와 유사도가 높은 사용자가 '4점자'라고 가정하자  
'4점자'의 영화 평점을 이용해 유사도로 가중 평균을 구한 값이 3점이 나왔다.  
이 경우 user_id인 '2점자'에게 3점을 그대로 부여하는 것은 위험하다.  
그 이유는 '4점자' 기준의 3점은 평균 이하지만 '2점자'기준 3점은 평균 이상이고,  
이는 추천 과정에서 의미가 변하게 되어버린 것이기 때문이다.  
  
그렇기 때문에 예측 평점을 구하는 과정에서,  
각 사용자의 평점 자체가 아닌 **평점의 평균 편차**를 이용해 **가중 평균**을 구한다.  
평점의 평균 편차는 '각 사용자가 movie_id에 부여한 평점 - 각 사용자의 전체 평점의 평균'으로 구할 수 있다.  
이를 **'평점 편차'**라고 정의한다  
  
위에서 정의한 **'평점 편차'**는  
각 사용자 평점의 평균 편차의 가중 평균, 즉 user_id가 해당 movie_id에 부여할 편차를 예상한 값이기 때문에  
user_id의 평균 점수에서 '평점 편차'를 더해준 값이 예상 평점이 된다.  
  
<코드 요약>  

>- 데이터 읽어오고 정리(RMSE 함수와 score(RMSE 사용)함수)하기
>- 각 사용자의 평점 편차 구해주기
>- def CF_knn_bias(user_id, movie_id, neighbor_size)
>    - 전달받은 movie_id에 대해 점수를 부여한 사람의 값이 rating_matrix에 존재할 때
>        - movie_id에 대한 평점이 없는 사람 제거하기
>        - 이웃의 크기가 1 이상일 때
>            - movie_id에 대해 평가한 사람이 1보다 클 때  
(prediction = np.dot(sim_scores, movie_ratings) / sim_scores.sum()  
prediction = prediction + rating_mean[user_id])  
>            (평균 편차를 이용해 '평점 편차'를 구한 후, user_id의 전체 평점 평균에 더해준다)
>            - movie_id에 대해 평가한 사람이 1이하일 때  
(prediction = rating_mean[user_id])(=user_id의 전체 평점 평균)
>        - 이웃의 크기가 0일때  
(prediction = np.dot(sim_scores, movie_ratings) / sim_scores.sum()  
prediction = prediction + rating_mean[user_id])  
>            (Neighbor size가 없기 때문에 전체 데이터를 이용해 계산해준다)
>    - 전달받은 movie_id에 대해 점수를 부여한 사람의 값이 rating_matrix에 존재하지 않을 때  
(prediction = rating_mean[user_id])(=user_id의 전체 평점 평균)
>- 예상 평점값 도출 (prediction)  
  
 --------------------------------------------------  
    
<가정>  
  
코드에 대한 이해를 돕기 위해 예시로,  
user_id = 2, movie_id = 2, neighbor_size = 30  
값을 넣는다고 가정하고 코드 실행 

## 1.1 데이터 읽어오고 정리하기  
사용해야하는 데이터를 읽어온다  
user에는 user_id에 대한 정보가 들어있고,  
movies에는 movie_id에 대한 정보,  
ratings에는 각 사용자들이 영화에 부여한 평점에 대한 정보가 들어있다 데이터 읽어오고 정리하기

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
# 데이터 읽어 오기 
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('C:/RecoSys/Data/u.user', sep='|', names=u_cols, encoding='latin-1')
i_cols = ['movie_id', 'title', 'release date', 'video release date', 'IMDB URL', 'unknown', 
          'Action', 'Adventure', 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 
          'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 
          'Thriller', 'War', 'Western']
movies = pd.read_csv('C:/RecoSys/Data/u.item', sep='|', names=i_cols, encoding='latin-1')
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('C:/RecoSys/Data/u.data', sep='\t', names=r_cols, encoding='latin-1')

ratings = ratings.drop('timestamp', axis=1)
movies = movies[['movie_id', 'title']]

In [4]:
users

,user_id,age,sex,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213
...,...,...,...,...,...
938,939,26,F,student,33319
939,940,32,M,administrator,02215
940,941,20,M,student,97229
941,942,48,F,librarian,78209


In [5]:
movies

,movie_id,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)
...,...,...
1677,1678,Mat' i syn (1997)
1678,1679,B. Monkey (1998)
1679,1680,Sliding Doors (1998)
1680,1681,You So Crazy (1994)


In [6]:
ratings

,user_id,movie_id,rating
0,196,242,3
1,186,302,3
2,22,377,1
3,244,51,2
4,166,346,1
...,...,...,...
99995,880,476,3
99996,716,204,5
99997,276,1090,1
99998,13,225,2


#### - train셋과 test셋 분리하기
**train데이터로 알고리즘**을 제작 후, **test로 정확도를 측정**해야 정확한 검증을 할 수 있다  
따라서 데이터를 train셋과 test셋으로 분리해준다

In [7]:
# train, test 데이터 분리
x = ratings.copy()
y = ratings['user_id']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, stratify=y)

In [8]:
# 정확도(RMSE)를 계산하는 함수 
def RMSE(y_true, y_pred):
    return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred))**2))

# 모델별 RMSE를 계산하는 함수 
def score(model, neighbor_size=0):
    id_pairs = zip(x_test['user_id'], x_test['movie_id'])
    y_pred = np.array([model(user, movie, neighbor_size) for (user, movie) in id_pairs])
    y_true = np.array(x_test['rating'])
    return RMSE(y_true, y_pred)

#### - full matrix 제작과 코사인유사도 구하기
train용으로 분리가 완료된 x_train 데이터를 full matrix로 구현해준다(=rating_matrix)  
rating_matrix를 이용해 코사인 유사도를 구해준다(=user_similarity)

- **rating_matrix (x_train의 full matrix)**

In [9]:
rating_matrix = x_train.pivot(index='user_id', columns='movie_id', values='rating')
rating_matrix

movie_id,1,2,3,4,5,6,7,8,9,10,...,1671,1672,1673,1674,1675,1676,1677,1678,1679,1681
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,NaN,NaN,NaN,NaN,4.0,1.0,5.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
940,NaN,NaN,NaN,2.0,NaN,NaN,4.0,5.0,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
941,5.0,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


- **user_similarity (rating_matrix 활용해 구한 코사인유사도)**

In [10]:
matrix_dummy = rating_matrix.copy().fillna(0)
user_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
user_similarity = pd.DataFrame(user_similarity, index=rating_matrix.index, columns=rating_matrix.index)
user_similarity

user_id,1,2,3,4,5,6,7,8,9,10,...,934,935,936,937,938,939,940,941,942,943
user_id,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.104283,0.030970,0.059271,0.326723,0.341355,0.351848,0.243820,0.056681,0.311995,...,0.280876,0.066749,0.200881,0.165332,0.154392,0.070048,0.205662,0.115544,0.149584,0.325238
2,0.104283,1.000000,0.070204,0.066799,0.038705,0.178330,0.068962,0.054493,0.107262,0.089341,...,0.107466,0.298464,0.290542,0.362215,0.269968,0.159824,0.141642,0.180067,0.082580,0.076437
3,0.030970,0.070204,1.000000,0.297498,0.027866,0.035307,0.065302,0.070619,0.056025,0.058989,...,0.000000,0.037415,0.127704,0.019895,0.098360,0.019007,0.137058,0.089539,0.053468,0.000000
4,0.059271,0.066799,0.297498,1.000000,0.043251,0.011453,0.056075,0.083992,0.074965,0.066411,...,0.046090,0.000000,0.011046,0.079860,0.075945,0.000000,0.153616,0.013692,0.067045,0.053161
5,0.326723,0.038705,0.027866,0.043251,1.000000,0.179064,0.286775,0.232548,0.073630,0.163670,...,0.265924,0.063973,0.059507,0.096046,0.126282,0.032626,0.203262,0.118360,0.119640,0.236429
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,0.070048,0.159824,0.019007,0.000000,0.032626,0.096299,0.084542,0.064595,0.051246,0.033491,...,0.029804,0.208761,0.211438,0.240207,0.356336,1.000000,0.094394,0.156781,0.040243,0.100846
940,0.205662,0.141642,0.137058,0.153616,0.203262,0.217194,0.237442,0.211892,0.078256,0.264550,...,0.259351,0.092907,0.146544,0.154381,0.147147,0.094394,1.000000,0.136978,0.215276,0.216583
941,0.115544,0.180067,0.089539,0.013692,0.118360,0.057959,0.067144,0.137659,0.141401,0.060104,...,0.022923,0.100764,0.287449,0.116343,0.359318,0.156781,0.136978,1.000000,0.105323,0.079485


#### - 각 사용자의 아이템별 평점 편차 구하기
각 사용자별로 평점의 부여하는 성향이 다르다.  
그렇기 때문에 평점 자체를 이용해 예상 평점을 구하는 것은  
전달 과정에서 의미가 변질될 리스크가 존재한다.  
따라서 리스크를 최소화 시키기 위해 평점 자체가 아닌, 각 항목별 평균 편차를 사용해준다

In [11]:
# rating_matrix의 각 user_id별 평균을 구해준다
rating_mean = rating_matrix.mean(axis=1)
rating_mean

user_id
1      3.583333
2      3.717391
3      2.875000
4      4.222222
5      2.885496
         ...   
939    4.297297
940    3.387500
941    3.875000
942    4.288136
943    3.436508
Length: 943, dtype: float64

In [12]:
# full matrix인 rating_matrix에서 rating_mean을 빼준다.
# 이것을 평점편차라고 부른다
rating_bias = (rating_matrix.T - rating_mean).T
rating_bias

movie_id,1,2,3,4,5,6,7,8,9,10,...,1671,1672,1673,1674,1675,1676,1677,1678,1679,1681
user_id,,,,,,,,,,,,,,,,,,,,,
1,1.416667,-0.583333,NaN,NaN,NaN,NaN,0.416667,-2.583333,1.416667,-0.583333,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.282609,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.717391,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,0.114504,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.702703,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
940,NaN,NaN,NaN,-1.3875,NaN,NaN,0.612500,1.612500,-0.387500,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
941,1.125000,NaN,NaN,NaN,NaN,NaN,0.125000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 1.2. def CF_knn_bias 설명
기본적으로 KNN 이웃 기반 협업 필터링을 활용했다.  
다만 예산 평점 계산에서 '평점 편차'의 개념이 들어간다.  
prediction = np.dot(sim_scores, movie_ratings) / sim_scores.sum() 의  
movie_ratings는 더이상 실제 평점이 아닌 '평점 편차'가 되고, prediction 역시 '예상 평점 편차'가 된다  
이에 따라, user_id의 전체 평점 평균 + '예상 평점 편차'를 해주어야  
실제 예상 평점값을 구할 수 있다

- user_id = 2, movie_id = 2, neighbor_size = 30 이라고 가정한다

In [13]:
user_id = 2
movie_id = 2
neighbor_size = 50

#### - 전달받은 movie_id가 rating_matrix에 존재하는지 확인이 필요하다
rating_matrix는 x_train을 활용해 만든 데이터 셋이다.  
따라서 탐색을 원하는 movie_id에 대한 정보가 들어있지 않을 수 있다  
만약 movie_id가 rating_matrix에 존재하지 않는다면(Null), 평균 점수로 정한 3.0점을 평점으로 부여한다
#### - 전달받은movie_id가 rating_matrix에 존재한다면, user_id에 대한 코사인 유사도와 movie_id에 대한 평점들을 받아온다
user_id와 다른 사용자의 코사인 유사도가 담긴 user_similarity를 추출해 sim_scores에 담아준다  
movie_id에 대해 다른 사용자들이 부여한 평점을 rating_matrix에서 추출해 movie_ratings에 담아준다

#### - movie_id에 대해 평가하지 않은 사용자의 정보는 사용 데이터에서 제외시켜준다
movie_id에 평점을 부여하지 않은 사용자들을  
movie_id의 평점 모음인 movie_ratings와 user_id와 다른 사용자간의 유사도 모음인 sim_scores에서 제외시켜준다

현재는 user_id = 2, movie_id = 2, neighbor_size = 30 이라고 가정하기 때문에  
movie_id에 대해 평점을 부여한 사람들의 정보(평점, 유사도)만을 받아준다 -> 'movie_ratings', 'sim_scores'

In [14]:
if movie_id in rating_bias:
    sim_scores = user_similarity[user_id].copy()
    movie_ratings = rating_bias[movie_id].copy()
    none_rating_idx = movie_ratings[movie_ratings.isnull()].index
    movie_ratings = movie_ratings.drop(none_rating_idx)
    sim_scores = sim_scores.drop(none_rating_idx)

- movie_ratings (movie_id에 대한 평점이 Null이 아닌 사람들의 **평점 모음**)

In [15]:
movie_ratings

user_id
1     -0.583333
5      0.114504
13    -0.109015
42     1.291971
49    -1.645963
         ...   
899   -0.514851
916   -0.319328
924   -0.741935
934    0.292308
943    1.563492
Name: 2, Length: 97, dtype: float64

- sim_scores (movie_id에 대한 평점이 Null이 아닌 사람들의 **유사도 모음**)

In [16]:
sim_scores

user_id
1      0.104283
5      0.038705
13     0.172017
42     0.094037
49     0.091039
         ...   
899    0.083221
916    0.106034
924    0.248887
934    0.107466
943    0.076437
Name: 2, Length: 97, dtype: float64

#### - Neighbor size(이웃의 크기)가 0인 경우와 1 이상인 경우
여기서 Neighbor size는 후에 'user_id와 유사도가 높은 [Neighbor size]개'에 사용된다 
만약 Neighbor size가 0이라면 유사도가 높은 0개의 유사도를 구하게 되고, 그것은 의미없는 값이다  
  
- 따라서 **Neighbor size가 0**일땐 범위를 따로 정하지 않고,  
사용자 범위를 전체로 삼아  
movie_id에 대한 평점 편차(movei_ratings)에 유사도를(sim_scores)를 가중치 삼아 '평점 편차의 가중평균'을 구해준다.  
그 후 user_id의 전체 평점 평균(rating_mean[user_id]) + '평점 편차의 가중평균'을 계산해 예상 평점에 넣어준다.
  
  
- 반대로 **Neighbor size가 1이상**이라면,  
sim_scores 관련 if문으로 이동하게 된다
  
  
#### - Neighbor size가 1 이상이면 그 후 sim_scores관련 if문이 등장한다
sim_scores는 평점을 부여한 사람들의 user_id에 대한 유사도 모음
sim_scores가 1이거나 그 이하인 경우는 연산에서 문제가 생긴다고 한다.  
  
- 따라서 **sim_score가 1이거나 그 이하**인 경우는 강제로  
user_id의 전체 평점 평균을 예상 평점값으로 넣어준다
  
  
- 반대로 **sim_scores가 1이상**인 경우에는  
사용자와 유사도가 높은 [Neighbor size]개를 범위로 잡는다  
movie_id에 대한 평점 편차(movei_ratings)에 유사도를(sim_scores)를 가중치 삼아 '평점 편차의 가중평균'을 구해준다.  
그 후 user_id의 전체 평점 평균(rating_mean[user_id]) + '평점 편차의 가중평균'을 계산해 예상 평점에 넣어준다.

현재는 user_id = 2, movie_id = 2, neighbor_size = 30 이라고 가정하고 있다  
Neighbor size는 0보다 크고, sim_scores 역시 1보다 큰 상태이다  
따라서 사용자와 **유사도가 높은 [Neighbor size]명**의 **'평점 편차의 가중평균'**을 구한 후  
user_id의 전체 평점 평균에 더해줘서 예상 평점을 구해준다

In [17]:
# 전달받은 movie_id에 대해 평가를 한 사람의 모음(sim_scores)의 수가 많은게 정상이지만,
# neighbor_size의 크기가 클 수도 있기 때문에 min 계산 후 작은 값을 선택해주기
neighbor_size = min(neighbor_size, len(sim_scores))

# 유사도의 상위 [Neighbor size]개 추출하기
sim_scores = np.array(sim_scores)
movie_ratings = np.array(movie_ratings)
user_idx = np.argsort(sim_scores)
sim_scores = sim_scores[user_idx][-neighbor_size:]
movie_ratings = movie_ratings[user_idx][-neighbor_size:]

# 선별된 유사도 상위 [Neighbor size]개의 가중 평균 구하기
# 영화의 평점 편차에 유사도를 가중치로 구해준다
prediction = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
print('평점편차:',prediction)
# 예측값에 user_id의 전체 평점 평균 더하기
prediction = prediction + rating_mean[user_id]
    
prediction

평점편차: -0.32624561448346445


3.391145689864362

코드 원본

In [18]:
def CF_knn_bias(user_id, movie_id, neighbor_size=0):
    if movie_id in rating_bias:
        sim_scores = user_similarity[user_id].copy()
        movie_ratings = rating_bias[movie_id].copy()
        none_rating_idx = movie_ratings[movie_ratings.isnull()].index
        movie_ratings = movie_ratings.drop(none_rating_idx)
        sim_scores = sim_scores.drop(none_rating_idx)   
        if neighbor_size == 0:
            prediction = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
            prediction = prediction + rating_mean[user_id]
        else:   
            if len(sim_scores) > 1:
                neighbor_size = min(neighbor_size, len(sim_scores))
                sim_scores = np.array(sim_scores)
                movie_ratings = np.array(movie_ratings)
                user_idx = np.argsort(sim_scores)
                sim_scores = sim_scores[user_idx][-neighbor_size:]
                movie_ratings = movie_ratings[user_idx][-neighbor_size:]
                prediction = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
                prediction = prediction + rating_mean[user_id]
            else:
                prediction = rating_mean[user_id]
    else:
        prediction = rating_mean[user_id]
    return prediction

prediction

3.391145689864362